In [1]:
import torch  # 导入PyTorch库
from torch import nn  # 从torch导入神经网络模块
from torch.utils.data import DataLoader  # 从torch.utils.data导入数据加载器
from torchvision import datasets  # 从torchvision导入数据集模块
from torchvision.transforms import ToTensor  # 从torchvision.transforms导入ToTensor转换

In [2]:
# 下载训练数据
training_data = datasets.FashionMNIST(
    root="data",  # 数据存储路径
    train=True,  # 选择训练集
    download=True,  # 如果数据不存在则下载
    transform=ToTensor(),  # 将数据转换为Tensor格式
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [3]:
# 下载测试数据
test_data = datasets.FashionMNIST(
    root="data",  # 数据存储路径
    train=False,  # 选择测试集
    download=True,  # 如果数据不存在则下载
    transform=ToTensor(),  # 将数据转换为Tensor格式
)

batch_size = 64  # 设置批次大小

In [4]:
# 创建数据加载器
train_dataloader = DataLoader(training_data, batch_size=batch_size)  # 训练数据加载器
test_dataloader = DataLoader(test_data, batch_size=batch_size)  # 测试数据加载器

In [5]:
# 遍历测试数据加载器，查看数据的形状
for X, y in test_dataloader:
    print(f"X的形状 [N, C, H, W]: {X.shape}")  # 输出 X 的形状
    print(f"y的形状: {y.shape}, 类型: {y.dtype}")  # 输出 y 的形状和类型
    break  # 只查看第一个批次

X的形状 [N, C, H, W]: torch.Size([64, 1, 28, 28])
y的形状: torch.Size([64]), 类型: torch.int64


In [6]:
# 获取用于训练的 CPU、GPU 或 MPS 设备
device = (
    "cuda"  # 如果有可用的 GPU，使用 cuda
    if torch.cuda.is_available()
    else "mps"  # 否则，使用 MPS
    if torch.backends.mps.is_available()
    else "cpu"  # 否则，使用 CPU
)

f"使用的设备: {device}"

'使用的设备: cuda'

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()  # 将输入展平
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),  # 第一层线性层，将28x28的图像输入到512维
            nn.ReLU(),  # ReLU 激活函数
            nn.Linear(512, 512),  # 第二层线性层，将512维输入到512维
            nn.ReLU(),  # ReLU 激活函数
            nn.Linear(512, 10)  # 输出层，将512维输入映射到10个类别
        )

    def forward(self, x):
        x = self.flatten(x)  # 展平输入
        logits = self.linear_relu_stack(x)  # 通过线性层处理输入
        return logits  # 返回 logits

model = NeuralNetwork().to(device)  # 创建模型并将其移动到指定设备
model  # 输出模型结构

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [8]:
loss_fn = nn.CrossEntropyLoss()  # 使用交叉熵损失
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)  # 使用随机梯度下降优化器

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # 数据集的大小
    model.train()  # 设置模型为训练模式
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # 将数据转移到指定设备

        # 计算预测误差
        pred = model(X)  # 通过模型进行预测
        loss = loss_fn(pred, y)  # 计算损失

        # 反向传播
        loss.backward()  # 计算梯度
        optimizer.step()  # 更新参数
        optimizer.zero_grad()  # 清除梯度

        if batch % 100 == 0:  # 每100个批次输出一次损失
            loss, current = loss.item(), (batch + 1) * len(X)
            f"损失: {loss:>7f}  [{current:>5d}/{size:>5d}]"  # 输出损失

In [1]:
# 定义测试函数，检查模型性能
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)  # 获取测试数据集的大小
    num_batches = len(dataloader)  # 获取批次数
    model.eval()  # 设置模型为评估模式（不计算梯度）
    
    test_loss, correct = 0, 0  # 初始化测试损失和正确预测的数量
    
    with torch.no_grad():  # 在此上下文中，不计算梯度
        for X, y in dataloader:  # 遍历测试数据加载器
            X, y = X.to(device), y.to(device)  # 将输入和标签转移到指定设备
            
            pred = model(X)  # 使用模型生成预测
            
            # 计算测试损失并累加
            test_loss += loss_fn(pred, y).item()  # 获取损失值并累计
            
            # 计算正确预测的数量
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # 将正确预测的数量累加

    # 计算平均测试损失
    test_loss /= num_batches
    # 计算正确预测的比例
    correct /= size
    
    # 输出测试结果，包括准确率和平均损失
    print(f"测试结果: \n准确率: {(100 * correct):>0.1f}%, 平均损失: {test_loss:>8f}\n")

# 训练过程分多次迭代（epoch）进行。在每个 epoch 中，模型会学习参数以进行更好的预测。
# 我们打印模型在每个 epoch 的准确率和损失；希望看到准确率增加，损失减少。

epochs = 5  # 设置训练的轮数为 5
for t in range(epochs):  # 遍历每个 epoch
    print(f"Epoch: {t + 1}/{epochs}\n" + "-" * 30)  # 输出当前的 epoch 号及总轮数
    
    # 初始化损失和正确预测数量
    total_loss = 0.0  # 累加损失
    total_correct = 0  # 正确预测数量
    total_samples = 0  # 总样本数量

    model.train()  # 设置模型为训练模式

    for batch_idx, (X, y) in enumerate(train_dataloader):  # 遍历训练数据加载器
        optimizer.zero_grad()  # 梯度清零
        
        pred = model(X.to(device))  # 前向传播，并将输入转移到指定设备
        loss = loss_fn(pred, y.to(device))  # 计算损失
        
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        
        # 累加损失和正确预测
        total_loss += loss.item()  # 添加当前批次损失
        total_samples += y.size(0)  # 更新总样本数量
        total_correct += (pred.argmax(1) == y.to(device)).sum().item()  # 更新正确的预测数量
        
        # 打印当前批次的损失
        if batch_idx % 100 == 0:  # 每 100 个 batch 打印一次
            print(f"批次 {batch_idx}, 损失: {loss.item():.6f}, 已处理样本: {batch_idx * len(X)}")

    # 计算训练准确率
    train_accuracy = 100 * total_correct / total_samples
    avg_loss = total_loss / len(train_dataloader)  # 计算平均损失
    print(f"训练损失: {avg_loss:.6f}, 训练准确率: {train_accuracy:.2f}%")  # 打印当前 epoch 的损失和准确率
            
    # 测试阶段
    test(test_dataloader, model, loss_fn)  # 调用测试函数，评估模型在测试集上的表现

print("完成!")  # 打印完成训练的提示

Epoch: 1/5
------------------------------


NameError: name 'model' is not defined

In [ ]:
"""保存模型的常用方法是序列化内部 state 字典（包含模型参数）"""
torch.save(model.state_dict(), "model.pth")
"已将 PyTorch 模型状态保存到 model.pth"

In [ ]:
"""加载模型的过程包括重新创建模型结构和加载 state 字典放入其中"""
model = NeuralNetwork().to(device)  # 创建模型并转移到指定设备
model.load_state_dict(torch.load("model.pth"))  # 加载模型状态字典